In [4]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPClassifier

def get_layer_sizes_from_clf(clf: MLPClassifier, X_sample=None):
    """
    Devuelve [n_in, h1, h2, ..., n_out].
    - Si el modelo NO está entrenado, usa clf.n_features_in_ (o X_sample.shape[1]) y clf.hidden_layer_sizes y 'n_outputs_' si existe.
    - Si está entrenado (coefs_), usa tamaños reales.
    """
    if hasattr(clf, "coefs_"):  # entrenado
        sizes = [clf.coefs_[0].shape[0]]
        for W in clf.coefs_:
            sizes.append(W.shape[1])
        return sizes
    else:
        if hasattr(clf, "n_features_in_"):
            n_in = clf.n_features_in_
        elif X_sample is not None:
            n_in = X_sample.shape[1]
        else:
            raise ValueError("Proporciona X_sample para deducir n_features_in_ si el modelo no está entrenado.")
        hidden = clf.hidden_layer_sizes
        if isinstance(hidden, int):
            hidden = [hidden]
        # Para salida no entrenado: estimar con n_classes_ si existe, si no solo mostramos capas ocultas
        n_out = getattr(clf, "n_outputs_", None) or getattr(clf, "n_classes_", None)
        if n_out is None:
            return [n_in, *hidden]  # sin salida conocida aún
        return [n_in, *hidden, int(n_out)]

def plot_mlp_architecture(clf: MLPClassifier, X_sample=None, title="Arquitectura MLP"):
    """
    Diagrama de cajas por capa con número de neuronas.
    Funciona ANTES o DESPUÉS de fit (si no está entrenado, pasa X_sample para deducir entrada).
    """
    sizes = get_layer_sizes_from_clf(clf, X_sample)
    n_layers = len(sizes)

    fig, ax = plt.subplots(figsize=(max(6, n_layers*2), 4))
    ax.set_title(title)
    ax.axis("off")

    # coordenadas básicas
    x_gap = 1.5
    y_base = 0.5
    box_w, box_h = 1.0, 0.6

    for i, n in enumerate(sizes):
        x = i * x_gap
        # caja
        rect = plt.Rectangle((x - box_w/2, y_base - box_h/2), box_w, box_h, fill=False)
        ax.add_patch(rect)
        # texto
        if i == 0:
            layer_name = "Entrada"
        elif i == n_layers - 1 and n_layers > 1:
            layer_name = "Salida"
        else:
            layer_name = f"Oculta {i}"
        ax.text(x, y_base + 0.1, f"{layer_name}", ha="center", va="center", fontsize=10)
        ax.text(x, y_base - 0.15, f"{n} neuronas", ha="center", va="center", fontsize=9)

        # flecha a la siguiente capa
        if i < n_layers - 1:
            x_next = (i+1) * x_gap
            ax.annotate("",
                        xy=(x_next - box_w/2, y_base),
                        xytext=(x + box_w/2, y_base),
                        arrowprops=dict(arrowstyle="->", lw=1))

    plt.show()

def plot_mlp_weights(clf: MLPClassifier, max_nodes_per_layer=16, max_edges=800, title="MLP (conexiones por peso)"):
    """
    Dibuja neuronas como puntos y muestra conexiones con grosor ~ |peso|.
    Requiere un MLPClassifier entrenado (usa clf.coefs_ y clf.intercepts_).
    - max_nodes_per_layer: limita neuronas dibujadas por capa para no saturar.
    - max_edges: limita número total de aristas (muestra una muestra uniforme si hay demasiadas).
    """
    if not hasattr(clf, "coefs_"):
        raise RuntimeError("Entrena el MLPClassifier antes de graficar pesos (faltan clf.coefs_).")

    coefs = clf.coefs_
    layer_sizes = [coefs[0].shape[0]] + [W.shape[1] for W in coefs]

    # posiciones de neuronas
    x_gap = 1.6
    xs = []
    ys = []
    node_indices = []  # mapping visible->(layer,i)
    for li, n in enumerate(layer_sizes):
        n_vis = min(n, max_nodes_per_layer)
        # indices a mostrar (muestra uniforme)
        if n <= n_vis:
            idxs = np.arange(n)
        else:
            idxs = np.linspace(0, n-1, n_vis).astype(int)
        x = li * x_gap
        # y distribuidos verticalmente
        if n_vis == 1:
            y_coords = np.array([0.0])
        else:
            y_coords = np.linspace(-0.8, 0.8, n_vis)
        xs.extend([x]*n_vis)
        ys.extend(y_coords.tolist())
        node_indices.append(idxs)

    fig, ax = plt.subplots(figsize=(max(6, len(layer_sizes)*2), 6))
    ax.set_title(title)
    ax.axis("off")

    # dibuja neuronas
    ax.scatter(xs, ys, s=50)

    # dibuja conexiones con grosor proporcional al peso
    # primero generamos lista de todas las aristas visibles
    edges = []
    for li, W in enumerate(coefs):
        src_idx = node_indices[li]
        dst_idx = node_indices[li+1]
        W_sub = W[np.ix_(src_idx, dst_idx)]
        # colección de (x1,y1,x2,y2, |w|)
        for i_s, si in enumerate(src_idx):
            for i_d, di in enumerate(dst_idx):
                w = W[si, di]
                edges.append((li, i_s, i_d, abs(float(w))))

    if not edges:
        plt.show()
        return

    # si hay demasiadas aristas, muestreamos
    if len(edges) > max_edges:
        step = int(np.ceil(len(edges) / max_edges))
        edges = edges[::step]

    # normaliza grosores
    weights = np.array([e[3] for e in edges])
    if weights.max() > 0:
        widths = 0.5 + 2.5 * (weights / weights.max())  # 0.5..3.0
    else:
        widths = np.full_like(weights, 0.5)

    # dibuja
    ptr = 0
    # reconstruye coordinates
    layer_offsets = np.cumsum([0] + [len(idxs) for idxs in node_indices[:-1]])
    for (li, i_s, i_d, _), lw in zip(edges, widths):
        # coords fuente
        x1 = li * x_gap
        n_vis_src = len(node_indices[li])
        y_srcs = np.linspace(-0.8, 0.8, n_vis_src) if n_vis_src > 1 else np.array([0.0])
        y1 = y_srcs[i_s]
        # coords destino
        x2 = (li+1) * x_gap
        n_vis_dst = len(node_indices[li+1])
        y_dsts = np.linspace(-0.8, 0.8, n_vis_dst) if n_vis_dst > 1 else np.array([0.0])
        y2 = y_dsts[i_d]
        ax.plot([x1, x2], [y1, y2], linewidth=float(lw))

    # etiquetas de capas
    for li, n in enumerate(layer_sizes):
        name = "Entrada" if li == 0 else ("Salida" if li == len(layer_sizes)-1 else f"Oculta {li}")
        ax.text(li * x_gap, 1.05, f"{name}\n({n})", ha="center", va="bottom", fontsize=10)

    plt.show()


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

df = pd.read_csv("datos_acelerometro.csv")

X = df[["ax", "ay", "az"]].values
y = df["label"].astype(str).values

Xtr, Xte, ytr, yte = train_test_split(X, y, test_size=0.2, random_state=0, stratify=y)

clf = MLPClassifier(hidden_layer_sizes=(100, 50), activation="relu", solver="adam", max_iter=300, random_state=0)
clf.fit(Xtr, ytr)

yp = clf.predict(Xte)
print("Accuracy:", accuracy_score(yte, yp))
